## Episodic Memory RAG

The episodic memory is a type of long-term memory that allows us to remember specific events or experiences in our lives. Unlike the declarative memory, which stores general knowledge and facts, the episodic memory is more focused on personal experiences and the context in which they occurred.

In the context of HybridAGI we wanted to ground this concept into Computer Sciences, and what best representation than a memory that store the program traces?

In HybridAGI's Trace Memory, each action is vectorized and stored, so the system can retrieve past actions between sessions by using the `PastActionSearch` tool.

Note: To avoid recursive recall of memories, we don't vectorize the `PastActionSearch` actions.

In [1]:
import hybridagi.core.graph_program as gp

# We first need to program our RAG Agent using a Graph Prompt Program
# So, let's create our program

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Action(
    id = "action_search",
    purpose = "Search for past steps to answer the Objective's question",
    tool = "PastActionSearch",
    prompt = "Use the Objective's question to infer the search query, ONLY two queries are authorized",
))

main.add(gp.Decision(
    id = "is_search_relevant",
    purpose = "Check if the steps field contains the answer to the Objective's question",
    question = """
Is the answer to the Objective's question in the steps field?
If the steps field is an empty list, it means that nothing have been found.
""",
))

main.add(gp.Action(
    id = "answer_context_based",
    purpose = "Answer the Objective's question",
    tool = "Speak",
    prompt = "Answer the Objective's question based on the previous steps in the context",
))

main.add(gp.Action(
    id = "answer",
    purpose = "Answer the Objective's question",
    tool = "Speak",
    prompt = "Answer the Objective's question based on your own knowledge",
))

main.connect("start", "action_search")
main.connect("action_search", "is_search_relevant")
main.connect("is_search_relevant", "answer_context_based", label="RELEVANT")
main.connect("is_search_relevant", "answer", label="EMPTY")
main.connect("is_search_relevant", "answer", label="NOTHING")
main.connect("answer_context_based", "end")
main.connect("answer", "end")

main.build() # Verify the graph program

print(main) # Print it to check


/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(action_search:Action {
  id: "action_search",
  purpose: "Search for past steps to answer the Objective's question",
  tool: "PastActionSearch",
  prompt: "Use the Objective's question to infer the search query, ONLY two queries are authorized"
}),
(is_search_relevant:Decision {
  id: "is_search_relevant",
  purpose: "Check if the steps field contains the answer to the Objective's question",
  question: "\n    Is the answer to the Objective's question in the steps field?\n    If the steps field is an empty list, it means that nothing have been found.\n    "
}),
(answer_context_based:Action {
  id: "answer_context_based",
  purpose: "Answer the Objective's question",
  tool: "Speak",
  prompt: "Answer the Objective's question based on the previous steps in the context"
}),
(answer:Action {
  id: "answer",
  purpose: "Answer the Objective's question",
  tool: "Speak",
  promp

In [2]:
# Now we can store it into memory and instanciate our agent

import dspy
from hybridagi.memory.integration.local import LocalProgramMemory, LocalTraceMemory
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.retrievers.integration.local import FAISSActionRetriever
from hybridagi.embeddings import SentenceTransformerEmbeddings
from hybridagi.modules.agents.tools import (
    SpeakTool,
    PastActionSearchTool,
)

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 384, # The dimention of the embeddings vector (also called dense vector)
)

program_memory = LocalProgramMemory(index_name="episodic_rag")

program_memory.update(main)

trace_memory = LocalTraceMemory(index_name="episodic_rag")

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state
    ),
    PastActionSearchTool(
        retriever = FAISSActionRetriever(
            trace_memory = trace_memory,
            embeddings = embeddings,
            distance = "cosine",
            max_distance = 0.9,
            k = 5,
            reranker = None,
        ),
    ),
]

rag_agent = GraphInterpreterAgent(
    program_memory = program_memory,
    trace_memory = trace_memory,
    embeddings = embeddings,
    agent_state = agent_state,
    tools = tools,
    debug= True,
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

# And call our agent

result = rag_agent(Query(query="What is a neuro-symbolic AGI?"))

print(result.final_answer)

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


--- Step 0 ---
Call Program: main
Program Purpose: What is a neuro-symbolic AGI?
--- Step 1 ---
Action Purpose: Search for past steps to answer the Objective's question
Action: {
  "queries": [
    "neuro symbolic AGI definition",
    "history of neuro-symbolic artificial general intelligence"
  ],
  "steps": []
}
--- Step 2 ---
Decision Purpose: Check if the steps field contains the answer to the Objective's question
Decision Question: 
    Is the answer to the Objective's question in the steps field?
    If the steps field is an empty list, it means that nothing have been found.
    
Decision: NOTHING
--- Step 3 ---
Action Purpose: Answer the Objective's question
Action: {
  "message": "A neuro-symbolic AGI, or Artificial General Intelligence, is a type of artificial intelligence that combines both neural networks (inspired by the structure and function of the human brain) and symbolic AI (which uses symbols and logic to represent and manipulate information). The goal is to create an

In [3]:
result = rag_agent(Query(query="What is the definition of a neuro-symbolic AGI?"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: What is the definition of a neuro-symbolic AGI?
--- Step 1 ---
Action Purpose: Search for past steps to answer the Objective's question
Action: {
  "queries": [
    "definition of neuro-symbolic AGI",
    "neuro-symbolic artificial general intelligence definition"
  ],
  "steps": [
    {
      "step": "--- Step 3 ---\nAction Purpose: Answer the Objective's question\nAction: {\n  \"message\": \"A neuro-symbolic AGI, or Artificial General Intelligence, is a type of artificial intelligence that combines both neural networks (inspired by the structure and function of the human brain) and symbolic AI (which uses symbols and logic to represent and manipulate information). The goal is to create an AGI that can learn from data, like a neural network, but also reason about and understand the world in a way similar to humans, like a symbolic AI. This approach aims to leverage the strengths of both paradigms to achieve a more general, flexible, a

In [4]:
# Let's have a look at the trace memory

trace_memory.show(notebook=True)